In [12]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn

In [13]:
X, y = load_boston(return_X_y=True)
y = y[..., np.newaxis]
y = np.concatenate((y, y * 2), axis=-1)
X = StandardScaler().fit_transform(X)

In [14]:
w = torch.ones(13, 2)
b = torch.zeros(2)
X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()

In [16]:
for i in range(10000):
    
    w.requires_grad = True
    b.requires_grad = True
    
    y__t = X_t @ w + b
    
#     y__t.requires_grad = True
    
    
    assert y__t.shape == y_t.shape
        
    l = 0.5 * torch.sum((y_t - y__t) ** 2) / len(y_t)
    l.backward()
    if i % 100 == 0:
        print(l)
#     print(b.grad)
    w = w.data - 0.1 * w.grad
    b = b.data - 0.1 * b.grad

    
    

tensor(1554.0861, grad_fn=<DivBackward0>)
tensor(55.4702, grad_fn=<DivBackward0>)
tensor(54.8817, grad_fn=<DivBackward0>)
tensor(54.7757, grad_fn=<DivBackward0>)
tensor(54.7478, grad_fn=<DivBackward0>)
tensor(54.7401, grad_fn=<DivBackward0>)
tensor(54.7379, grad_fn=<DivBackward0>)
tensor(54.7373, grad_fn=<DivBackward0>)
tensor(54.7372, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>)
tensor(54.7371, grad_fn=<DivBackward0>

In [5]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.ones([in_dim, out_dim])
#         self.w = np.random.randn(in_dim, out_dim)
        self.b = np.zeros(out_dim)
        self.dw = None
        self.db = None
        self.in_dim = self.w.shape[0]
        self.out_dim = self.w.shape[1]

        
    def forward(self, x):
        self.x = x
        return np.matmul(x, self.w) + self.b
    
    def backward(self, d):
        
        J = np.zeros([self.x.shape[0], self.out_dim, np.prod(self.w.shape)])
        j = 0
        for i in range(self.out_dim):
            J[:, i: i + 1, j: j + self.in_dim] = self.x[:, np.newaxis, :]
            j += self.in_dim
        
        
        self.db = np.mean(d, 0)
        assert self.db.shape == self.b.shape, (d.shape, self.db.shape, self.b.shape)
        
        d = d[:, :, np.newaxis]
        J = np.swapaxes(J, -2, -1)

        self.dw = np.reshape(np.mean(J @ d, axis=0), self.w.shape, order='F')
        assert self.dw.shape == self.w.shape, (self.dw.shape, self.w.shape)
                
#         J = np.repeat(self.w[np.newaxis, ...], self.x.shape[0], axis=0)
#         d = J @ d.T
        return d        
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [6]:
class MeanSquaredError:
    def __init__(self):
        self.y_ = None
    def forward(self, y_, y):
        assert y_.shape == y.shape
        self.y_ = y_
        self.y = y
#         print(y_)
        l = 0.5 * np.sum(np.square(y - y_)) / len(y)
        return l
    
    def backward(self):
        d = -(self.y - self.y_)
        return d

In [7]:
# class Linear:
#     def __init__(self, in_dim, out_dim):
#         self.w = np.random.randn(in_dim, out_dim)
#         self.b = np.zeros([out_dim])
#         self.dw = None
#         self.db = None
#         self.x = None
        
#     def forward(self, x):
#         self.x = x
#         return np.matmul(x, self.w) + self.b
    
#     def backward(self, d):
#         self.dw = np.matmul(self.x.T, d)
#         self.db = np.sum(d, axis=0)
#         assert self.dw.shape == self.w.shape
#         assert self.db.shape == self.b.shape

#         d = d @ self.w.T
#         return d
        
#     def step(self, lr):
#         self.w = self.w - lr * self.dw
#         self.b = self.b - lr * self.db

In [8]:
class LinearRegression:
    def __init__(self):
        self.linear = Linear(13, 2)
        self.loss = MeanSquaredError()
        
    def forward(self, x, y):
        x = self.linear.forward(x)
        loss = self.loss.forward(x, y)
        return loss
    
    def backward(self):
        d = self.loss.backward()
        self.linear.backward(d)
    
    def step(self, lr):
        self.linear.step(lr)
        

In [9]:
# X = np.array([[1, 2, 3], [1, 2, 3]])
# y = np.array([[4], [4]])

In [10]:
linreg = LinearRegression()

In [11]:
for i in range(100000):
    loss = linreg.forward(X, y)
    if i % 100 == 0:
        print(loss, 'loss')
    linreg.backward()
    linreg.step(0.1)


1554.0859822096998 loss
55.470173610926885 loss
54.88169658632446 loss
54.77571618031153 loss
54.74782663407481 loss
54.74008197277605 loss
54.73791795418272 loss
54.737312853873746 loss
54.73714364265141 loss
54.737096323706865 loss
54.73708309122181 loss
54.73707939082898 loss
54.73707835603414 loss
54.73707806665936 loss
54.73707798573726 loss
54.737077963107836 loss
54.737077956779636 loss
54.73707795500999 loss
54.73707795451513 loss
54.73707795437673 loss
54.73707795433804 loss
54.73707795432722 loss
54.737077954324185 loss
54.73707795432334 loss
54.7370779543231 loss
54.737077954323034 loss
54.73707795432302 loss
54.737077954323006 loss
54.737077954323006 loss
54.737077954323006 loss
54.73707795432302 loss
54.737077954323006 loss
54.73707795432301 loss
54.737077954323006 loss
54.737077954323006 loss
54.737077954323006 loss
54.73707795432301 loss
54.73707795432301 loss
54.737077954323006 loss
54.737077954323006 loss
54.73707795432301 loss
54.73707795432301 loss
54.73707795432301 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/david/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-bf5140d92fe1>", line 5, in <module>
    linreg.backward()
  File "<ipython-input-8-d5bc70f4b7ee>", line 13, in backward
    self.linear.backward(d)
  File "<ipython-input-5-40a92187aedc>", line 21, in backward
    J[:, i: i + 1, j: j + self.in_dim] = self.x[:, np.newaxis, :]
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/david/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  

KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X, y)

In [ ]:
np.sum(0.5 * (lr.predict(X) - y) ** 2) / len(X)

In [ ]:
linreg.linear.forward(X).sum()

In [ ]:
lr.predict(X).sum()